### RandomForest Best Model  
- RandomizedSearchCV : params :{'max_leaf_nodes': 23, 'max_depth': 7, 'criterion': 'entropy'} / roc_auc_score_val : 0.8605244741486768

- GridSearchCV: params:{'criterion': 'entropy','max_depth': 13,'max_leaf_nodes': 34,'n_estimators': 200} / roc_auc_score_val:0.8605244741486768

  =>RandomizedSearchCV 에서 찾은 'max_leaf_nodes': 23 와 'max_depth': 7 이 들어 갈 수있는 범위를 만들어 준다 
  
- GridSearchCV: params:{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 200})/roc_auc_score_val :0.865253524994359


  =>앞선 GridSearchCV 에서 max_leaf_nodes 가 최고 점인 34가 나왔음으로 nodes 가 제한이 없는 모델을 다시 만들어 최적의 값을 구해 본다.

In [1]:
import pandas as pd 
import numpy as np
data =pd.read_csv("preprocessing\cs_data.csv",
                 header=0,
                 index_col=0)

In [2]:
# NA를 전부 없앤 값 = datap
data.dropna(inplace=True)
data.head()

SeriousDlqin2yrs  RevolvingUtilizationOfUnsecuredLines  age  \
1                 1                              0.766127   45   
2                 0                              0.957151   40   
3                 0                              0.658180   38   
4                 0                              0.233810   30   
5                 0                              0.907239   49   

   NumberOfTime30-59DaysPastDueNotWorse  DebtRatio  MonthlyIncome  \
1                                     2   0.802982         9120.0   
2                                     0   0.121876         2600.0   
3                                     1   0.085113         3042.0   
4                                     0   0.036050         3300.0   
5                                     1   0.024926        63588.0   

   NumberOfOpenCreditLinesAndLoans  NumberOfTimes90DaysLate  \
1                               13                        0   
2                                4                        0   
3                                2                        1   
4                                5                        0   
5                                7                        0   

   NumberRealEstateLoansOrLines  NumberOfTime60-89DaysPastDueNotWorse  \
1                             6                                     0   
2                             0                                     0   
3                             0                                     0   
4                             0                                     0   
5                             1                                     0   

   NumberOfDependents  
1                 2.0  
2                 1.0  
3                 0.0  
4                 0.0  
5                 0.0

In [3]:
#Data 로 X,y 구분 
X= data.drop(columns='SeriousDlqin2yrs')
y= data['SeriousDlqin2yrs']
X.shape , y.shape

((120269, 10), (120269,))

In [4]:
# 1.Dataset 구분 TRAIN, VAL, TEST
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split (X, y, test_size =0.2, stratify=y ,random_state=0) 
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=0)

In [7]:
# 2.Scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler
sc= StandardScaler()
X_train_scaled1=sc.fit_transform(X_train)
X_val_scaled1=sc.transform(X_val)
X_test_scaled1=sc.transform(X_test)

In [ ]:
######## Random Forest best model 찾기 ############ 

In [65]:
#RandomizedSearchCV 
from sklearn.model_selection import RandomizedSearchCV, train_test_split
rf= RandomForestClassifier(random_state=0)
param ={
    "max_depth":range(3,11), # 1~10
    "max_leaf_nodes":range(3,31,5), #[3,6,9,.......,27,30]
    "criterion":["gini","entropy"]
}

rs= RandomizedSearchCV(rf, # 모델
                      param, # 파라미터 후보
                      n_iter=10,  # 총 조합 중에 10개를 random하게 선택하서 테스트
                      cv =5,
                      scoring='roc_auc',# cross validation 의 fold 수
                      n_jobs =-1
                      )

In [66]:
rs.fit(X_train_scaled1, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=0),
                   n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': range(3, 11),
                                        'max_leaf_nodes': range(3, 31, 5)},
                   scoring='roc_auc')

In [67]:
rs.best_score_ , rs.best_params_

(0.8478683553615106,
 {'max_leaf_nodes': 23, 'max_depth': 7, 'criterion': 'entropy'})

In [68]:
best_model =rs.best_estimator_
best_model

RandomForestClassifier(criterion='entropy', max_depth=7, max_leaf_nodes=23,
                       random_state=0)

In [69]:
#RandomizedSearchCV 로 찾은 best roc_auc rf 
from sklearn.metrics import roc_curve, plot_roc_curve, roc_auc_score
roc_auc_score(y_val, best_model.predict_proba(X_val_scaled1)[:,1])

0.8605244741486768

In [ ]:
#RandomizedSearchCV score : 0.8478683553615106/ params :{'max_leaf_nodes': 23, 'max_depth': 7, 'criterion': 'entropy'} / roc_auc_score_val : 0.8605244741486768

In [13]:
#GridSearchCV
from sklearn.model_selection  import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
param ={
        'n_estimators': [100,200], #200 이 베스트
        'max_depth': range(7,15), # 7이 베스트 RandomizedSearchCV
        'max_leaf_nodes':range(20,35), # 23이 베스트 RandomizedSearchCV
        'criterion':["gini","entropy"]
}

rf= RandomForestClassifier(random_state=0)
gs = GridSearchCV(rf, param, scoring ='roc_auc', cv=5, n_jobs=-1)

In [14]:
gs.fit(X_train_scaled1, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(7, 15),
                         'max_leaf_nodes': range(20, 35),
                         'n_estimators': [100, 200]},
             scoring='roc_auc')

In [15]:
gs.best_score_ , gs.best_params_

(0.8492543007948189,
 {'criterion': 'entropy',
  'max_depth': 13,
  'max_leaf_nodes': 34,
  'n_estimators': 200})

In [16]:
best_model1 =gs.best_estimator_
best_model1

RandomForestClassifier(criterion='entropy', max_depth=13, max_leaf_nodes=34,
                       n_estimators=200, random_state=0)

In [17]:
#GridSearchCV 로 찾은 best roc_auc rf 
from sklearn.metrics import roc_curve, plot_roc_curve, roc_auc_score
roc_auc_score(y_val, best_model1.predict_proba(X_val_scaled1)[:,1])

0.862239279822552

In [ ]:
#GridSearchCV RandomForestClassifier(criterion='entropy', max_depth=13, max_leaf_nodes=34,n_estimators=200, random_state=0)

In [8]:
#GridSearchCV
from sklearn.model_selection  import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
param ={
        'n_estimators': [100,200], #200 이 베스트
        'max_depth': range(7,15), # 7이 베스트 RandomizedSearchCV
        'criterion':["gini","entropy"]
}

rf= RandomForestClassifier(random_state=0)
gs1 = GridSearchCV(rf, param, scoring ='roc_auc', cv=5, n_jobs=-1)

In [9]:
gs1.fit(X_train_scaled1, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(7, 15),
                         'n_estimators': [100, 200]},
             scoring='roc_auc')

In [10]:
gs1.best_score_ , gs1.best_params_

(0.8503665474164366,
 {'criterion': 'gini', 'max_depth': 10, 'n_estimators': 200})

In [11]:
best_model1 =gs1.best_estimator_
best_model1

RandomForestClassifier(max_depth=10, n_estimators=200, random_state=0)

In [12]:
#GridSearchCV 로 찾은 best roc_auc rf 
from sklearn.metrics import roc_curve, plot_roc_curve, roc_auc_score
roc_auc_score(y_val, best_model1.predict_proba(X_val_scaled1)[:,1])

0.865253524994359

In [ ]:
#GridSearchCV score : (0.8503665474164366, {'criterion': 'gini', 'max_depth': 10, 'n_estimators': 200})/roc_auc_score_val :0.865253524994359